In [1]:

%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np

import tifffile
from matplotlib import pyplot as plt


import plotly.express as px

from scipy.spatial.distance import cdist, pdist

# from pombelength import cell_length
from timelapse import tracking
from timelapse.tracking import trackcells, length_dataframe
from timelapse.imageutils import label_mask, make_movie
%matplotlib notebook

### Load image data

Right now this is assuming a projected tiff file. This is easy to change.

* data - the image data in a numpy array
* predicted - the mask prediction from deep learning in a numpy array

In [2]:
# imagefile = 'Data/3-17-20_WT-Agar-ON_bestz_3channel.tif'
imagepath = "path/to/movie"
imagefile = imagepath + "/max_project_movie.tif"
data = tifffile.imread(imagefile)
predicted = tifffile.imread('/path/to/inferred.tif')
results_folder = tracking.make_results_dir(imagefile)
results_folder

'Results/max_project_movie'

### Load data from processing fluoresence and transmitted light

In [ ]:
df = pd.read_pickle('/path/to/dataframe/pickle/.pkl')
df = df.reset_index()
df['minutes'] = 10*df['frame']
df.columns

### Create a labeled image of the predicted masks


In [25]:
labeled = predicted

In [26]:
length_df = tracking.get_cell_lengths(results_folder, labeled)

In [27]:
xdf = trackcells(length_df, frame_col='frame',
                    xm='cmx', ym='cmy', max_separation=10, max_lost=2)

In [28]:
'''
this creates a dictionary with the key being the cellid and the values the
row index for the fluorescence dataframe and length dataframe

key : int
    the cellid
    
value : dict
    key : 'f' or 'length'
    value : int
        row index of 
'''

cell_dict = tracking.cellid_row_table(df, xdf, labeled)

In [106]:
'''
Enter the x, y coordinates of a nucleus to follow the cell through
multiple divisions
'''
x = 1378
y = 1170

cellx = int(x)
celly =  int(y)

frame = 1


cellid, trace_df = tracking.track_cell(df, xdf, cell_dict, labeled, cellx, celly, frame)
cellid, cellx, celly

(187, 1378, 1170)

In [ ]:
%matplotlib inline

trace_df = trace_df[trace_df['frame'] < 33]
tracking.plot_cell_trace(trace_df, "rdna_vol_ratio")
#plt.ylim((0, 3000))

In [ ]:
snaps = tracking.plot_timelapse(trace_df, data, cellx, celly, "length", results_folder)

In [11]:
import plotly.express as px
import plotly.graph_objects as go
import dash

In [12]:
import base64
import io
#import cv2
from PIL import Image

def image_to_jpeg(frame, to_bgr=True) -> bytes:
    frame = frame if not to_bgr else cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    _, jpeg = cv2.imencode(".png", frame)
    return jpeg.tobytes()

def image_to_jpeg_b64(frame, to_bgr=True) -> bytes:
    img = Image.fromarray(frame)
    buffer = io.BytesIO()
    img.save(buffer, format='PNG')
    buffer.seek(0)
    
    #return base64.b64encode(image_to_jpeg(frame, to_bgr=to_bgr)).decode("utf-8")
    return base64.b64encode(buffer.getvalue()).decode("utf-8")

In [13]:
x = tracking.to_rgb(data[0, ...])
_x = 255*x
_x = _x.astype(np.uint8)

In [ ]:


go_image = go.Image(source=f"data:image/png;base64,{image_to_jpeg_b64(_x)}")
fig = go.FigureWidget([go_image])
fig.update_layout(width=800, height=800)

pdict = {}
plist = []

def clicker(trace, points, state):
    pdict[0] = points
    plist.append(points)
    plist.append(trace)
    print(points)
    
go_image.on_click(clicker)
fig.show()